# Modeling Crop Yield: Landsat + Sentinel
## Python modules

In [ ]:
import warnings
import time
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import geopandas

import pyarrow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from scipy.stats import pearsonr

import math
import seaborn as sns

## Satellite Parameters

 - For a description of the Landsat 8 mission, see the US Geological metadata [here.]()
 - For a description of the Sentinel 2 mission, see the US Geological metadata [here.]()

We'll use **ls8** and **sn2** for Landsat 8 and Sentinel 2 missions, respectively, throughout this notebook to denote satellite specific parameters. 

In [ ]:
ls8_satellite = "landsat-8-c2-l2"
sn2_satellite = "sentinel-2-l2a"

### Choose band combination.

 - For a description of **Landsat 8** bands, see the [US Geological Survey documentation here.](https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites)
 - For a description of **Sentinel 2** bands, see the [US Geological Survey documentation here.](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-sentinel-2#:~:text=4%20bands%20at%2010%20meter,%2Dinfrared%20(842%20nm)

According to our results, bands **(insert band selection here)** result in the best model performance for Landsat, and **(insert band selection here)** result in the best model performance for Sentinel for the task of predicting maize yields in Zambia.

In [ ]:
# Landsat bands
ls8_bands = "1-2-3-4-5-6-7"


# Sentinel bands
# sn2_bands = "2-3-4"
sn2_bands = "2-3-4-8"
# sn2_bands = "2-3-4-5-6-7-8-11-12"

### Choose the number of points that were featurized.

Each value in the following chunk represents the amount of thousands of points that were featurized in each respective feature file. These points represent a uniform subset of the spatial grid of Zambia. Points are spaced at uniform intervals for each selection, measured in kilometers in the longitudinal direction for each set of features. Selecting a greater quantity of points results in a denser spatial sample and increases the spatial resolution of the model. Regardless of the quantity of points selected, each point is buffered by the same distance, resulting in a 1km^2 cell around each point.

Selection remains the same for Landsat and Sentinel.

In [ ]:
# points = "4"
points = "15"
# points = "24"
# points = "42"

#### Choose to keep only areas with crops (`True`) or to keep all points (`False`)

Selecting `True` applies a "cropland mask" to the spatial grid of Zambia. This retains only the regions of the country in which maize is  grown, according to the **(insert source here)**. As a result, the spatial extent of the features that are fed into the model are highly subset for the specific task at hand: modeling maize yields. According to our results, selecting `True` **(insert increases or decreases here)** model performance.

Selecting `False` results in modeling with the maximum spatial extent of the features, with more generalized features as a result.

In [ ]:
# crop_mask = True
crop_mask = False

#### Impute NA values by descending group levels (True) or `scikit learn`'s simple imputer (False)

Imputing "manually" by descending group levels imputes NA values in multiple "cascading" steps, decreasing the proportion of inoutated values with each step. First, the NA values are imputed at by both `year` and `district`, which should yield imputed values that most closely match the feature values that would be present in the data if there was no clouds obscuring the satellite images. Next, the remaining NA values that could not be imputed by both `year` and `district` are imputed by only `district`. Lastly, the remaining NA vlaues that could not be imputed by both `year` and `district` or by just `district` are imputed by `year` only. This option gives the user more control and transparency over how the imputation is executed.

Imputing using `scikit learn`'s simple imputer executes standard imputation, the details of which can be found in the `scikitlearn` documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [ ]:
impute_manual = True
# impute_manual = False

Choose a weighted average (`True`) or a simple mean (`False`) to use when collapsing features to administrative boundary level.

In [ ]:
# weighted_avg = True
weighted_avg = False

### Choose which months to use in the model.

Note that months 10, 11, and 12 get pushed to the next year because the growing season (November - May) spans the calendar year. Maize is planted in November, starts to change color with maturity in May, and is harvested in June - August. According to our results, subsetting the months to **(insert month selection here)** increases model performance.

Selection remains the same for Sentinel and Landsat

In [ ]:
# month_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# month_range = [      3, 4, 5, 6, 7, 8, 9            ]
month_range = [         4, 5, 6, 7, 8, 9            ]
# month_range = [            5, 6, 7, 8, 9            ]
# month_range = [         4, 5, 6, 7, 8               ]
# month_range = [            5, 6, 7, 8               ]

### Unchanging parmaters

The parameters in the following chunk are set for the country of Zambia for with 1000 features, regardless of the satellite selected. The start years for each satellite reflect the respective years that Landsat 8 and Sentinel 2A missions began.

The number of features is set to 1000 to serve as a staple parameter among the several other parameters varied during the model optimization process. Changing this parameter in the following code chunk will result in an error because featurizing landsat imagery for a different number of features was outside the scope of this project.

In [ ]:
# set home directory
taylor_data_dir = "/capstone/cropmosaiks/data"  

# set plot sizes
if points == "4":
    marker_sz = 60
elif points == "15":
    marker_sz = 15
elif points == "24":
    marker_sz = 10
else:
    marker_sz = 8

In [ ]:
country_code = "ZMB"
num_features = 1000

# set start years for both satellites
ls8_year_start = 2013 # Landsat
sn2_year_start = 2015 # Sentinel
year_end = 2018       # same for both satellites

# set file paths
ls8_feature_file_name = (f'{ls8_satellite}_bands-{ls8_bands}_{country_code}_{points}k-points_{num_features}-features')
sn2_feature_file_name = (f'{sn2_satellite}_bands-{sn2_bands}_{country_code}_{points}k-points_{num_features}-features')
weight_file_name = (f'{country_code}_crop_weights_{points}k-points')

## Administrative boundaries 

Administrative boundaries reflect the **(insert number of districts in dataset)** district boundaries within the country of Zambia. A district can be likened to a state within the larger U.S.A. We subset the spatial grid to district level becuase the crop yield data is at the district level of specificity. The features are originally produced at higher spatial resolution, then summarized to the district level in order to train the model with ground-truth crop data. 

In [ ]:
country_shp = geopandas.read_file(f'{taylor_data_dir}/boundaries/gadm36_{country_code}_2.shp')
country_shp = country_shp.rename(columns = {'NAME_2': 'district'})[['district', 'geometry']]
country_shp.district = country_shp.district.replace("MPongwe", 'Mpongwe', regex=True)
country_districts = country_shp.district.sort_values().unique().tolist()
country_shp = country_shp.set_index('district')
country_shp.shape
country_shp.plot(linewidth = 1, edgecolor = 'black' )
# country_shp.plot()

## Crop yield

Zambian maize yield data reflects the predicted annual maize yield provided by farmers in the month of May, when the maize matures and changes colors prior to harvest, which allows the farmers to estimate what their yield will be in the following months. These predictions are in units of metric tons per hectare and provide valuable insight to the Zambian government as they plan for the quanitites of food to import into the country in the future. For more metadata, see the websites for the [Central Statistics Office of Zambia (CSO)](https://www.zamstats.gov.zm/) and the [Summary statistics from CSO.](https://www.zamstats.gov.zm/agriculture-environment-statistics/)

In order to standardize the names of all districts shared between the geoboundaries and the crop yield data, we correct for spelling, dashes, and apostrophes. 


In [ ]:
crop_df = pd.read_csv(taylor_data_dir+'/crops/cfs_maize_districts_zambia_2009_2018.csv')
crop_df.district = crop_df.district.replace(
    {"Itezhi-tezhi": 'Itezhi-Tezhi',
     "Kapiri-Mposhi": 'Kapiri Mposhi',
     "Shang'ombo": 'Shangombo',
     "Chienge": 'Chiengi'
    }, regex=True)
crop_districts = crop_df.district.sort_values().unique().tolist()
crop_df = crop_df[['district', 'year', 'yield_mt']]
ln = len(crop_df[crop_df.year == 2016].district)
crop_df = crop_df.set_index('district')
ln
# crop_df

In [ ]:
list(set(crop_districts) - set(country_districts))

In [ ]:
list(set(country_districts) - set(crop_districts))

In [ ]:
country_crop = geopandas.GeoDataFrame(crop_df.join(country_shp), crs = country_shp.crs)

## Crop land

In [ ]:
weights = pd.read_feather(f"{taylor_data_dir}/weights/{weight_file_name}.feather")
# weights

In [ ]:
weights_gdf = geopandas.GeoDataFrame(
    weights, 
    geometry = geopandas.points_from_xy(x = weights.lon, y = weights.lat), 
    crs='EPSG:4326'
)
weights_gdf.plot(figsize = (10,10),
                 cmap = 'inferno',
                 markersize = marker_sz,
                 alpha = .9,
                 column = 'crop_perc')
# plt.axis('off')

In [ ]:
weights.crop_perc = weights.crop_perc.fillna(0)
# #weights.crop_perc = weights.crop_perc + 0.0001

## Features

Append annual features files together into one file: `features_raw`.

#### Landsat 8

In [ ]:
features_ls8_raw = geopandas.GeoDataFrame()

for yr in range(ls8_year_start, year_end + 1):
    print(f"Opening: {ls8_feature_file_name}_{yr}.feather")
    features_ls8 = pd.read_feather(f"{taylor_data_dir}/features/{ls8_satellite}/{ls8_feature_file_name}_{yr}.feather")
    
    if (yr == 2013):
        features_ls8 = features_ls8[features_ls8.month > 9]
    else:
        pass
    
    # concatenate the feather files together, axis = 0 specifies to stack rows (rather than adding columns)
    features_ls8_raw = pd.concat([features_ls8_raw, features_ls8], axis=0)
    
    print("feature.shape", features_ls8_raw.shape)
    print("Appending:", yr)
    print("")

#### Sentinel 2

In [ ]:
features_sn2_raw = geopandas.GeoDataFrame()

for yr in range(sn2_year_start, year_end + 1):
    print(f"Opening: {sn2_feature_file_name}_{yr}.feather")
    features_sn2 = pd.read_feather(f"{taylor_data_dir}/features/{sn2_satellite}/{sn2_feature_file_name}_{yr}.feather")
    
    if (yr == 2015):
        features_sn2 = features_sn2[features_sn2.month > 9]
    else:
        pass
    
    # concatenate the feather files together, axis = 0 specifies to stack rows (rather than adding columns)
    features_sn2_raw = pd.concat([features_sn2_raw, features_sn2], axis=0)
    
    print("feature.shape", features_sn2_raw.shape)
    print("Appending:", yr)
    print("")

In [ ]:
# Create copies of both feature datasets
features_ls8 = features_ls8_raw.copy()
features_sn2 = features_sn2_raw.copy()

We want to carry the months October, November, and December over to the following year's date. These months represent the start of the growing season for the following year's maize yield

In [ ]:
# Landsat
features_ls8['year'] = np.where(
    features_ls8['month'].isin([10, 11, 12]),
    features_ls8['year'] + 1, 
    features_ls8['year'])

features_ls8 = features_ls8[features_ls8['year'] <= year_end]
features_ls8.sort_values(['year', 'month'], inplace=True)


# Sentinel
features_sn2['year'] = np.where(
    features_sn2['month'].isin([10, 11, 12]),
    features_sn2['year'] + 1, 
    features_sn2['year'])

features_sn2 = features_sn2[features_sn2['year'] <= year_end]
features_sn2.sort_values(['year', 'month'], inplace=True)

### Filter month range

In [ ]:
# subset the features to only the month range selected at the top of the notebook
features_ls8 = features_ls8[features_ls8.month.isin(month_range)]
features_sn2 = features_sn2[features_sn2.month.isin(month_range)]

### Pivot wider
Here we pivot the data from long format to wide by indexing on 'lon', 'lat', 'year', 'month' and using the unstack function. We then map column names based on the month index and the associated features so month '01' is appended to each feature for that month making 0_01, 1_01 etc. This results in a Tidy data structure, with each row representing an image, and each column representing a feature for a certain month.

In [ ]:
# Landsat
features_ls8 = features_ls8.set_index(['lon','lat', "year", 'month']).unstack()
features_ls8.columns = features_ls8.columns.map(lambda x: '{}_{}_ls8'.format(*x))

# Sentinel
features_sn2 = features_sn2.set_index(['lon','lat', "year", 'month']).unstack()
features_sn2.columns = features_sn2.columns.map(lambda x: '{}_{}_sn2'.format(*x))

In [ ]:
features_sn2.head(50)

### Join Landsat & Sentinel dataframes

In [ ]:
features = features_ls8.join(features_sn2, how = 'left')

### Replace "inf" values with `NaN`

Infinity values are the result of **(insert reason here)**. We replace them with `NaN` because **(insert reason here)**.

In [ ]:
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features = features.reset_index()
# features

### Attach crop weights
Attach weight to each point (% area cropped of surrounding 1 km^2).

In [ ]:
features = features.join(weights.set_index(['lon', 'lat']), on = ['lon', 'lat'])
features = features.drop(["geometry"], axis = 1)
# features

### Mask croppped regions
Any 1 km^2 cell with a crop percentage > 0 will be retained.\
The mask will not be applied if `crop_mask` is set to `False` at the top of this notebook

In [ ]:
if crop_mask:
    features = features[features.crop_perc > 0]
else:
    pass
# features

### Make "features" a `GeoDataFrame`

The coordinate reference system is set to **EPSG 4326 - WGS 84**, the latitude/longitude coordinate system based on the Earth's center of mass, used by the Global Positioning System.

In [ ]:
features = geopandas.GeoDataFrame(
    features, 
    geometry = geopandas.points_from_xy(x = features.lon, y = features.lat), 
    crs='EPSG:4326'
)

### Plot any single features

In [ ]:
# mn = 7
# yr = 2017
# sat = 'sn2' # ls8
# feature = 854

# features[features.year == yr].plot(
#     column = f"{feature}_{mn}_{sat}",
#     figsize = (10,10),
#     marker='H',
#     # legend = True,
#     markersize = marker_sz,
# )

### Drop 'lat' and 'lon' columns

In [ ]:
# Drop the redundant independent lon and lat columns now that they are in a geometry column
features = features.drop(['lon', 'lat'], axis = 1)

### Join features to country geometry

In [ ]:
features = features.sjoin(country_shp, how = 'left', predicate = 'within')
# features

### Correct column names and drop geometry

In [ ]:
features = (
    features
    .dropna(subset=['index_right'])
    .rename(columns = {"index_right": "district",})
    .reset_index(drop = True)
)
points = features.copy()
points = features[['geometry']]
features = features.drop(['geometry'], axis = 1)
# features

### Impute missing values

Imputing "manually" by descending group levels imputes NA values in multiple "cascading" steps, decreasing the proportion of inoutated values with each step. First, the NA values are imputed at by both `year` and `district`, which should yield imputed values that most closely match the feature values that would be present in the data if there was no clouds obscuring the satellite images. Next, the remaining NA values that could not be imputed by both `year` and `district` are imputed by only `district`. Lastly, the remaining NA vlaues that could not be imputed by both `year` and `district` or by just `district` are imputed by `year` only. This option gives the user more control and transparency over how the imputation is executed.

Imputing using `scikit learn`'s simple imputer executes standard imputation, the details of which can be found in the `scikitlearn` documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

The imputation approach depends on the selection made at the top of this notebook for `impute_manual`.

In [ ]:
# compute the number of cells in the features dataframe, based on the amount of rows (images), months, and feature columns
num_cells = len(features) * len(month_range) * num_features
num_cells

In [ ]:
%%time
if impute_manual:
    print(f'Starting with\n{(features.isna().sum().sum() / num_cells)*100:.02f} % NA Values')
    features = (
        features
        .fillna(features
                .groupby(['year', 'district'], as_index=False)
                .transform('mean')
               )
    )
    print(f'\nFilling NA values with year district group average\n{(features.isna().sum().sum() / num_cells)*100:.02f} % NA Values')
    features = (
        features
        .fillna(features
                .groupby(['district'], as_index=False)
                .transform('mean')
               )
    )
    print(f'\nRemaining NA values\n{(features.isna().sum().sum() / num_cells)*100:.02f} % NA Values')
else:
    features = features.set_index(['year', 'district'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features)
    features[:] = imputer.transform(features)
    features = features.reset_index()

In [ ]:
features.dropna(axis=1)

### Summarise to administrative boundary level
Weighted by cropped area, or simple mean, depending on the selection at the top of this notebook for `weighted_avg`. 

In [ ]:
var_cols = features.columns[1:-2].values.tolist()
features.columns[1:-2]

In [ ]:
%%time
if weighted_avg:
    features_summary = (
        features
        .groupby(['year', 'district'], as_index=False)
        .apply(lambda x: pd.Series([sum(x[v] * x.crop_perc) / sum(x.crop_perc) for v in var_cols]))
    )
else:
    features_summary = features.groupby(['district',"year"], as_index = False).mean()
# features_summary

### Join crop data

In [ ]:
crop_df_x = crop_df[crop_df.year >= sn2_year_start + 1]
crop_df_x = crop_df_x[~crop_df_x.index.isin(['Mafinga', 'Ikelenge'])]
crop_df_x.reset_index(inplace=True)
crop_df_x.head()

In [ ]:
features_summary = (
    features_summary
    .set_index(["district", "year"])
    .join(other = crop_df_x.set_index(["district", "year"]))
    .reset_index())
#features_summary

## Model

In [ ]:
model_year = features_summary[features_summary.year.isin([
   2016,
   2017,
   2018,
])]

### Define `x's` and `y's`

In [ ]:
x_all = model_year.drop([
    'district', 
    'year', 
    'yield_mt',
    "crop_perc"
], axis = 1)

# y_all = features_summary.yield_mt
y_all = np.log10(model_year.yield_mt.to_numpy() + 1)

In [ ]:
len(x_all)

In [ ]:
len(y_all)

### Standardize Features
We will use the default configuration and scale values to subtract the mean to center them on 0.0 and divide by the standard deviation to give the standard deviation of 1.0. First, a StandardScaler instance is defined with default hyperparameters.

In [ ]:
scalar = StandardScaler().fit_transform(x_all)
x_all = pd.DataFrame(scalar)
#x_all[3].hist()

y_all = StandardScaler().fit_transform(pd.DataFrame(y_all))
#y_all.hist()

### Split into train and test sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=0
)

In [ ]:
print("Total N: ", len(x_all), "\n", 
      "Train N: ", len(x_train), "\n",
      "Test  N: ", len(x_test), sep = "")

In [ ]:
x_all.isna().sum()

In [ ]:
x_all.head()

### Train model
Now that our data has been standardized, we can use the same penalazation fac

In [ ]:
# ridge_cv_random = RidgeCV(cv=5, alphas=np.logspace(-8, 8, base=10, num=17))
# ridge_cv_random.fit(x_train, y_train)

### Validation set $R^2$ performance

In [ ]:
print(f"Validation R2 performance {ridge_cv_random.best_score_:0.2f}")

### Save copy of completed data 

In [ ]:
features_copy = features.copy()
features_copy['geometry'] = points.geometry

## Save output

In [ ]:
features.to_feather(f'{ls8_satellite}_{sn2_satellite}_"combined".feather')